# Obteniendo el número de muertes confirmadas y vacunados contra COVID-19 por semana epidemiológica de cada uno de los 24 departamentos del Perú 

## 0. Cargar librerías

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from epiweeks import Week

import functions as fn

## 1. Cargar direcciones de RawData

In [2]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

## 2. Procesar del dataset de fallecidos

### 2.1. Leer el dataset

In [3]:
# Leer solamente la columna fecha de fallecimiento y departamento
fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO']
df_fal =  pd.read_csv(fal_url, sep = ';', usecols = fal_col, dtype = {'FECHA_FALLECIMIENTO':'int32', 'DEPARTAMENTO': 'category'})
del fal_col

# Transformar a formato fecha
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')
df_fal.head()

,FECHA_FALLECIMIENTO,DEPARTAMENTO
0,2021-06-11,LAMBAYEQUE
1,2021-03-17,PIURA
2,2021-06-02,ICA
3,2021-07-03,AREQUIPA
4,2021-05-06,LIMA


### 2.2. Procesar el dataset

Se añaden 3 columnas. El año, la semana epidemiológica correspondiente y un contador para contar cada caso de fallecimiento por COVID-19. Se elimina la columna **'fecha_fallecimiento'**.

In [4]:
# Agregamos el año y semana epidemilógica de cada fallecido
fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO')
                 
df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8'})

del df_fal['FECHA_FALLECIMIENTO']
df_fal.head()

,DEPARTAMENTO,year,epi_week
0,LAMBAYEQUE,2021,23
1,PIURA,2021,11
2,ICA,2021,22
3,AREQUIPA,2021,26
4,LIMA,2021,18


**IMPORTANTE:** No tenemos el total de fallecidos por departamento y semana epidemiológica, sino el departamento y semana epidemiológica de cada caso.

### 2.3. Realizar una crosstab de fallecidos por COVID-19 por departamento de Perú, para cada año y semana epidemiológica

Esto se lo realiza para obtener el TOTAL de fallecidos por cada departamento y semana epidemiológica

In [21]:
falxdep = pd.crosstab(index=[df_fal['year'], df_fal['epi_week']],
                      columns=[df_fal['DEPARTAMENTO']],
                      margins = False)

falxdep = falxdep.add_suffix('_fal')
falxdep

DEPARTAMENTO   AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                   0           0             0             0   
     11                   0           0             0             0   
     12                   0           0             0             0   
     13                   0           4             0             0   
     14                   0           0             0            15   
...                     ...         ...           ...           ...   
2021 45                   2          10             2             7   
     46                   1          11             3             7   
     47                   1          12             3             8   
     48                   3           8             2             9   
     49                   1           4             0             4   

DEPARTAMENTO   AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                   0              0           0          0   
     11                   0              0           0          0   
     12                   1              0           0          1   
     13                   0              0           2          3   
     14                   0              2          15          2   
...                     ...            ...         ...        ...   
2021 45                   6              3           4          5   
     46                   6              2           8          4   
     47                   2              4          12          4   
     48                   8              2           9          6   
     49                   3              3           7          3   

DEPARTAMENTO   HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_fal  \
year epi_week                                 ...               
2020 10                       0            0  ...           1   
     11                       0            0  ...           1   
     12                       0            0  ...           3   
     13                       0            0  ...           6   
     14                       0            0  ...           9   
...                         ...          ...  ...         ...   
2021 45                       4            2  ...           0   
     46                       2            2  ...           1   
     47                       4            1  ...           2   
     48                       1            2  ...           0   
     49                       1            4  ...           0   

DEPARTAMENTO   MADRE DE DIOS_fal  MOQUEGUA_fal  PASCO_fal  PIURA_fal  \
year epi_week                                                          
2020 10                        0             0          0          0   
     11                        0             0          0          1   
     12                        0             0          0          1   
     13                        1             0          0          2   
     14                        0             0          0          9   
...                          ...           ...        ...        ...   
2021 45                        1             3          3         24   
     46                        2             2          1         32   
     47                        1             1          1         25   
     48                        1             1          1         38   
     49                        0             0          1         14   

DEPARTAMENTO   PUNO_fal  SAN MARTIN_fal  TACNA_fal  TUMBES_fal  UCAYALI_fal  
year epi_week                                                                
2020 10               0               2          0           0            0  
     11               0               0          0           0            0  
     12               0               0          0           0            2  
     

Ahora tenemos el número de fallecidos por cada departamento y semana epidemiológica

## 3. Procesar el dataset de vacunados

No existe una manera directa de calcular el total de personas completamente vacunadas por departamento. Para lograrlo se prevee lo siguiente:

- El dataset de vacunados (RawData/TB_VACUNACION_COVID19.csv) solo proporciona información sobre el centro de vacunación llamado **'id_centro_vacunacion'**. NO el departamento.

- El dataset de los centros de vacunación [(RawData/TB_CENTRO_VACUNACION.csv)]() se puede utilizar para hacer **"match"** del **'id_centro_vacunacion'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica de 0 a 1894 que representa a cada distrito.

- Finalmente, con el dataset de UBIGEO [(RawData/TB_UBIGEOS.csv)]() es posible reemplazar a cada **'id_ubigeo'** con el departamento respectivo.

### 3.1. Leer el dataset


In [8]:
# Leemos solo la columna dosis del dataset de vacunados usando dask
vac_col = ['id_eess', 'dosis', 'fecha_vacunacion']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'id_eess':'int32','dosis': 'int8', 'fecha_vacunacion':'int32'})

# Déjamos solamente a personas con 3 o menos dosis
df_vac = df_vac[df_vac['dosis'] == 2]
del df_vac['dosis']

# Convertimos a formato fecha
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

del vac_col
df_vac.head()

,fecha_vacunacion,id_eess
0,2021-07-01,18536
1,2021-07-03,4482
3,2021-11-26,24328
13,2021-10-28,23890
16,2021-11-20,17753


### 3.2. Procesar el dataset

In [9]:
# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')

df_vac.head()

,id_eess,year,epi_week
0,18536,2021,26
1,4482,2021,26
3,24328,2021,47
13,23890,2021,43
16,17753,2021,46


### 3.3. Procesar los datasets de **UBIGEO** y **CENTROS DE VACUNACIÓN**.

#### 3.3.1. Cargar direcciones de RawData

In [10]:
ubigeo_url = 'RawData/TB_UBIGEOS.csv'
vaccenter_url = 'RawData/TB_EESS.csv'

#### 3.3.2. Procesar los datasets

Para el dataset de UBIGEO solo se necesita las columnas 'id_ubigeo' y 'departamento'. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas 'id_centro de vacunación' e 'id_ubigeo'.

In [12]:
# Leemos los datasets de ubigeo y centros de vacunación
ubigeo = pd.read_csv(ubigeo_url, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(vaccenter_url, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

vaccenter.head()

,id_eess,departamento
0,17241,LORETO
1,14145,LORETO
2,7083,LORETO
3,549,LORETO
4,3207,LORETO


Ahora con el nuevo dataset **(vaccenter)** es posible encontrar el departamento correspondiente a 'id_centro de vacunacion' del dataset de vacunados **(df_vac)**.

#### 3.3.3. Encontrar el departamento correspondiente de todas las personas vacunadas

In [13]:
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')
del df_vac['id_eess']
del vaccenter

df_vac.head()

,year,epi_week,departamento
0,2021,26,LIMA
1,2021,26,LIMA
2,2021,47,CAJAMARCA
3,2021,43,LIMA
4,2021,46,PIURA


### 3.4. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [14]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['departamento']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [15]:
vac_dose = crosstab4dask(df_vac)
vac_dose = vac_dose.add_suffix('_vac')
vac_dose

departamento   AMAZONAS  ANCASH  APURIMAC  AREQUIPA  AYACUCHO  CAJAMARCA  \
year epi_week                                                              
2021 6                0       0         0         0         0          0   
     7                0       0         0         0         0          0   
     8                0       2         0         0         0          0   
     9              887    1975      2496      3763      2362       2726   
     10             529    1083       833      4584      2804        939   
     11            2726    5544      2813      6467      1424       5080   
     12             352    1551       492      2038       776       2252   
     13             170     328       371       338       242        403   
     14             175     547       328      2864       339        535   
     15              96    4566       130     11601       127        226   
     16             121     447       194       571       399        214   
     17             396    3013       124       374       871       2010   
     18              21     842         9       737       170        526   
     19            1164     184        15        36        32       1154   
     20              15    6813        14       405      1126       3416   
     21              33    6835      1423     11546        91       5397   
     22              72      93       721      8510        46       6505   
     23            6679   33053     14382     22349     17987      16399   
     24            9206   15900      5634     46264      8736      15494   
     25            2898    3845      5409     17030      8060      23147   
     26            1626   25410      2022     27805      3212       8974   
     27            1546    6465      1280     28196      6832       5985   
     28            3541   18437      3641     70345      3840      10227   
     29            3940   12939      7313      8675      2531       8506   
     30            4498   26942      4685     27857     11707      39864   
     31            9443   40758      9814     25813      8650      36674   
     32           12771   39700     22122      8033      7179      23565   
     33            9089   23584      7837     16186      8544      10449   
     34            8953   14366      5078     41384      6787      17591   
     35            3162   13915      4117     53343     12483      32141   
     36            2568   29867      4999      4673     11988      34908   
     37            8663   24541      6768     52653     11703      24659   
     38            9743   12673     11791     12349     15808      14224   
     39            9921   83060     13913     55502     17700      79474   
     40           19558   97339     20635     87026     19986      67548   
     41           11002   38440     18342     75989     20537      35407   
     42           11179   32226     11599      6630     16315      43165   
     43            8261   15712      9861     14436     10762      30514   
     44            4019    8903      7550     19051      7864      20028   
     45            3831   14119      9002    101712     11392      14857   
     46            4427   17634      8753     26200      8370      17576   
     47           13326   49689     14422     43767      6275      45037   
     48            9085   41958     13279     17107      6799      37153   
     49            2259    7710      6045      8623      4191      15472   

departamento   CALLAO  CUSCO  HUANCAVELICA  HUANUCO  ...  LORETO  \
year epi_week                                        ...           
2021 6              0      0             0        0  ...       0   
     7              0      1             0        0  ...       0   
     8              0      0             0        1  ...       1   
     9           4639   3204           888     2032  ...    1752   
     10          5445   3905          1621      181  ...    1262   
     11          4609   305

In [27]:
falvac_dep = pd.concat([falxdep, vac_dose], axis = 1).fillna(0)
falvac_dep

AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                   0           0             0             0   
     11                   0           0             0             0   
     12                   0           0             0             0   
     13                   0           4             0             0   
     14                   0           0             0            15   
...                     ...         ...           ...           ...   
2021 45                   2          10             2             7   
     46                   1          11             3             7   
     47                   1          12             3             8   
     48                   3           8             2             9   
     49                   1           4             0             4   

               AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                   0              0           0          0   
     11                   0              0           0          0   
     12                   1              0           0          1   
     13                   0              0           2          3   
     14                   0              2          15          2   
...                     ...            ...         ...        ...   
2021 45                   6              3           4          5   
     46                   6              2           8          4   
     47                   2              4          12          4   
     48                   8              2           9          6   
     49                   3              3           7          3   

               HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_vac  \
year epi_week                                 ...               
2020 10                       0            0  ...         0.0   
     11                       0            0  ...         0.0   
     12                       0            0  ...         0.0   
     13                       0            0  ...         0.0   
     14                       0            0  ...         0.0   
...                         ...          ...  ...         ...   
2021 45                       4            2  ...     16101.0   
     46                       2            2  ...     14096.0   
     47                       4            1  ...     26282.0   
     48                       1            2  ...     42490.0   
     49                       1            4  ...      6520.0   

               MADRE DE DIOS_vac  MOQUEGUA_vac  PASCO_vac  PIURA_vac  \
year epi_week                                                          
2020 10                      0.0           0.0        0.0        0.0   
     11                      0.0           0.0        0.0        0.0   
     12                      0.0           0.0        0.0        0.0   
     13                      0.0           0.0        0.0        0.0   
     14                      0.0           0.0        0.0        0.0   
...                          ...           ...        ...        ...   
2021 45                   1771.0        1837.0     2706.0    40989.0   
     46                   2283.0        1878.0     3011.0    35067.0   
     47                   1466.0        4326.0     3189.0    46063.0   
     48                   4074.0        3006.0     3975.0    57173.0   
     49                   1908.0        1077.0     1997.0    17197.0   

               PUNO_vac  SAN MARTIN_vac  TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                                                
2020 10             0.0             0.0        0.0         0.0          0.0  
     11             0.0             0.0        0.0         0.0          0.0  
     12             0.0             0.0        0.0         0.0          0.0  
     13             

In [28]:
falvac_dep.to_csv('Data/vacunados_y_fallecidos_x_semanaEpi_x_departamentos.csv')